In [4]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
import dlib

In [5]:
detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [6]:
data = pd.read_csv('data/fer2013.csv')

In [116]:
N = 1000
D = data
data.groupby('Usage').count()

,emotion,pixels
Usage,,
PrivateTest,3589,3589
PublicTest,3589,3589
Training,28709,28709


In [117]:
training_image = []
training_label_full = []
test_image = []
test_label_full = []


for index, row in D.iterrows():
    if row['Usage'] == 'Training':
        training_image.append(np.reshape(np.array(row['pixels'].split(), dtype=np.uint8), (48, 48)))
        training_label_full.append(row['emotion'])
    else:
        test_image.append(np.reshape(np.array(row['pixels'].split(), dtype=np.uint8), (48, 48)))
        test_label_full.append(row['emotion'])

In [118]:
def get_landmarks(_image):
    image = _image.copy()
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
                
#         xmean = np.mean(xlist) #Get the mean of both axes to determine centre of gravity
#         ymean = np.mean(ylist)
        xmean = xlist[29]
        ymean = ylist[29]
        xcentral = [(x-xmean) for x in xlist] #get distance between each point and the central point in both axes
        ycentral = [(y-ymean) for y in ylist]
        
        # point 29 - nose tip
        # point 26 - middle point b/w two eyes
        cv.circle(image, (int(xlist[29]), int(ylist[29])), 1, (0,0,255))
        
        angle_nose = np.arctan2((ylist[26]-ymean), (xlist[26]-xmean)) * 180 / np.pi
        if angle_nose < 0:
            angle_nose += 90
        else:
            angle_nose -= 90
            
#         print(angle_nose)

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x) #Add the coordinates relative to the centre of gravity
            landmarks_vectorised.append(y)

            #Get the euclidean distance between each point and the centre point (the vector length)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)

            #Get the angle the vector describes relative to the image, corrected for the offset that the nosebrigde has when the face is not perfectly horizontal
            anglerelative = (np.arctan2((z - ymean), (w - xmean)) * 180 / np.pi) - angle_nose
            landmarks_vectorised.append(anglerelative)
        
    if len(detections) < 1: 
        landmarks_vectorised = "error"     
    
    return landmarks_vectorised

In [119]:
import time

def make_sets():
    training_data = []
    test_data = []
    training_label = []
    test_label = []
    
    #Append data to training and prediction list, and generate labels 0-7
    index = 0
    
    start_time = time.time()
    
    for item in training_image:
        # image = cv.imread(item) #open image
        # gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #convert to grayscale
        # clahe_image = clahe.apply(gray)
#         print(index)
        landmarks_vectorised = get_landmarks(item)
        if landmarks_vectorised == "error":
            pass
        else:
            training_data.append(landmarks_vectorised) #append image array to training data list
            training_label.append(training_label_full[index])
            
        index += 1
        
    print('train - ', time.time() - start_time)
    
    index = 0
    for item in test_image:
        # image = cv.imread(item)
        # gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        # clahe_image = clahe.apply(gray)
#         print(index)
        landmarks_vectorised = get_landmarks(item)
        if landmarks_vectorised == "error":
            pass
        else:
            test_data.append(landmarks_vectorised)
            test_label.append(test_label_full[index])
            
        index += 1
        
    print('test - ', time.time() - start_time)

    return training_data, test_data, training_label, test_label

# --------------------------------------------------------

train_data, test_data, training_label, test_label = make_sets()

print(len(train_data), len(training_label))

# pd.DataFrame(train_data)
# pd.DataFrame(test_data)

('train - ', 95.29250979423523)
('test - ', 117.99933099746704)
(19863, 19863)


In [120]:
from sklearn import svm

clf = svm.SVC(kernel='rbf', gamma=2)

# print(len(training_label))
clf.fit(train_data, training_label)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=2, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [128]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 20), random_state=1)

nn_clf.fit(train_data, training_label)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(10, 20), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [129]:
# res = nn_clf.predict(test_data, test_label)

res = pd.DataFrame(res)

res.to_csv('out_nn_clf')

TypeError: predict() takes exactly 2 arguments (3 given)